In [3]:
from collections import Counter
import dill
import dill
data_path = '../data/records_final.pkl'
voc_path = '../data/voc_final.pkl'
ehr_adj_path = '../data/ehr_adj_final.pkl'
ddi_adj_path = '../data/ddi_A_final.pkl'

ehr_adj = dill.load(open(ehr_adj_path, 'rb'))
ddi_adj = dill.load(open(ddi_adj_path, 'rb'))
data = dill.load(open(data_path, 'rb'))
voc = dill.load(open(voc_path, 'rb'))
diag_voc, pro_voc, med_voc = voc['diag_voc'], voc['pro_voc'], voc['med_voc']

split_point = int(len(data) * 2 / 3)
data_train = data[:split_point]
eval_len = int(len(data[split_point:]) / 2)
data_test = data[split_point:split_point + eval_len]
data_eval = data[split_point+eval_len:]

In [4]:

def most_common_medical(data, voc, choice=0, top=10):
    c = Counter()
    cnt = 0 
    for patient in data:
        for adm in patient:
            for item in adm[choice]:
                c[item] += 1
                cnt += 1
    for letter, count in c.most_common(top):
        print('%s: %.4f' % (letter, count/cnt))



def case_study_statics(data, adm, adm_ja, ):
    # y_gt: multi-hot
    c = Counter()

In [6]:
most_common_medical(data, diag_voc, choice=2, top=10)

4: 0.0580
5: 0.0545
8: 0.0534
21: 0.0503
3: 0.0468
6: 0.0458
2: 0.0438
14: 0.0366
17: 0.0275
28: 0.0269


In [ ]:
most_common_medical(data, med_voc, choice=2, top=10)

# low score w.r.t Jaccard (Medication)

In [35]:
import dill
import os
from collections import Counter
# model_name = 'Retain_small'
model_name = 'NEW_GAMENet_small_neg'
case_study = dill.load(open(os.path.join('saved', model_name, 'case_study.pkl'), 'rb'))

# sort
sort_case_study = [ case_study[k] for k in sorted(case_study.keys())] 

In [50]:
sort_case_study[-3]

{'ja': 0.8,
 'patient': [[[825, 7, 25],
   [65],
   [4, 6, 0, 14, 19, 1, 26, 3, 28, 39, 2, 12, 15, 87, 32]],
  [[825, 43, 24, 25],
   [602],
   [4, 6, 12, 3, 0, 14, 19, 2, 32, 39, 28, 1, 33, 15]]],
 'y_label': [[0, 1, 2, 3, 4, 6, 12, 14, 19, 32, 39],
  [0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 28, 32, 39, 87]]}

In [55]:
# EHR
import dill
ddi_A = dill.load(open('../data/ddi_A.pkl','rb'))
records = [[[[825, 7, 25],
   [65],
   [4, 6, 0, 14, 19, 1, 26, 3, 28, 39, 2, 12, 15, 87, 32]],
  [[825, 43, 24, 25],
   [602],
   [4, 6, 12, 3, 0, 14, 19, 2, 32, 39, 28, 1, 33, 15]]]]
# test ddi in ground truth set

all_cnt = 0
dd_cnt = 0
for patient in records:
    for adm in patient:
        med_code_set = adm[2]
        for i, med_i in enumerate(med_code_set):
            for j, med_j in enumerate(med_code_set):
                if j <= i:
                    continue
                all_cnt += 1
                if ddi_A[med_i, med_j] == 1 or ddi_A[med_j, med_i] == 1:
                    dd_cnt += 1
                    print('i,j',med_i,med_j)
print(dd_cnt / all_cnt)

i,j 4 6
i,j 4 0
i,j 4 14
i,j 4 3
i,j 4 28
i,j 4 2
i,j 4 15
i,j 4 32
i,j 6 19
i,j 6 3
i,j 6 28
i,j 6 12
i,j 0 3
i,j 0 28
i,j 0 12
i,j 0 15
i,j 19 3
i,j 19 2
i,j 1 12
i,j 26 2
i,j 3 28
i,j 3 2
i,j 3 12
i,j 3 87
i,j 3 32
i,j 28 12
i,j 2 87
i,j 12 15
i,j 4 6
i,j 4 3
i,j 4 0
i,j 4 14
i,j 4 2
i,j 4 32
i,j 4 28
i,j 4 15
i,j 6 12
i,j 6 3
i,j 6 19
i,j 6 28
i,j 12 3
i,j 12 0
i,j 12 28
i,j 12 1
i,j 12 15
i,j 3 0
i,j 3 19
i,j 3 2
i,j 3 32
i,j 3 28
i,j 0 28
i,j 0 15
i,j 19 2
0.27040816326530615


In [92]:
# EHR
import dill
from collections import defaultdict
ddi_A = dill.load(open('../data/ddi_A_final.pkl','rb'))
# records = [[[0, 1, 2, 3, 4, 6, 12, 14, 19, 32, 39],[0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 28, 32, 39, 87]]]
records = [[[0, 1, 2, 3, 4, 6, 12, 14, 15, 16, 19, 26, 32, 34, 39, 48, 59, 73],
[0, 1, 2, 3, 4, 5, 6, 9, 11, 12, 14, 15, 19, 22, 26, 28, 34, 42, ]] ]
# test ddi in ground truth set

all_cnt = 0
dd_cnt = 0
ddi_set = defaultdict(set)
for patient in records:
    for adm in patient:
        med_code_set = adm
        for i, med_i in enumerate(med_code_set):
            for j, med_j in enumerate(med_code_set):
                if j <= i:
                    continue
                all_cnt += 1
                if ddi_A[med_i, med_j] == 1 or ddi_A[med_j, med_i] == 1:
                    dd_cnt += 1
                    ddi_set[med_i].add(med_j)
print(dd_cnt / all_cnt)
print(ddi_set)

0.0915032679738562
defaultdict(<class 'set'>, {0: {26}, 1: {32, 26, 42}, 2: {48}, 12: {34, 73, 22, 26, 28}, 15: {32, 48, 34, 19}, 19: {26, 34}, 26: {34, 59}, 34: {73}})


v1 'diag' '1912', '2724', '2749' [Malignant neoplasm of brain, Hyperlipidmia, Gout]
v2 'diag' '1912', '3485', '2720', '2749' [Malignant neoplasm of brain, Cerebral Edema, Hypercholesterolemia, Gout]
records = [[[[825, 7, 25],
   [65],
   [0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 26, 28, 32, 39, 87]],
  [[825, 43, 24, 25],
   [602],
   [0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 28, 32, 33, 39]]]]
N02B, A01A, A02B, A06A, B05C, A12C, C07A, C02D, N02A, B01A, C10A, J01D, N03A, A04A, H04A  
0.0663
{0: {26}, 1: {32, 26}, 12: {26, 28}, 15: {32, 19}, 19: {26}, 32: {33}}

# GAMENet 0.0342 
[[0, 1, 2, 3, 4, 6, 12, 14, 19, 32, 39],
  [0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 28, 32, 39, 87]]
  {1: {32}, 12: {28}, 15: {32, 19}}
 

# RETAIN 0.1240
[0, 1, 2, 3, 4, 5, 6, 8, 11, 12, 15, 19, 25, 26, 32]
true [0,1,2,3,4,6,12,15,19,32] 
{0: {26}, 1: {32, 26}, 2: {25}, 3: {25}, 12: {25, 26}, 15: {32, 19}, 19: {25, 26}, 25: {32, 26}}

# Nearest 0.0761
[0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 26, 28, 32, 39, 87] 
[0,1,2,3,4,6,12,14,15,19,28,32,39]
{0: {26}, 1: {32, 26}, 12: {26, 28}, 15: {32, 19}, 19: {26}}

# Linear
[0, 1, 3] 0


# DMNC 0.0915
true[[0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 26, 32] + 6,
[0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 32, 39]] + 7
[[0, 1, 2, 3, 4, 6, 12, 14, 15, 16, 19, 26, 32, 34, 39, 48, 59, 73],
[0, 1, 2, 3, 4, 5, 6, 9, 11, 12, 14, 15, 19, 22, 26, 28, 34, 42, ]] 
{0: {26}, 1: {32, 26, 42}, 2: {48}, 12: {34, 73, 22, 26, 28}, 15: {32, 48, 34, 19}, 19: {26, 34}, 26: {34, 59}, 34: {73}})

# Leap 0.0468
[[0, 2, 3, 6, 12, 19, 26, 39], [0, 1, 2, 3, 4, 6, 7, 19, 87]]
{0: {26}, 12: {26}, 19: {26}})



In [64]:
import numpy as np
sorted(np.array([14, 2, 8, 6, 5, 21, 3, 4, 7, 24, 28, 16, 17, 11, 36, 44, 13, 51, 70, 30])-2)

[0, 1, 2, 3, 4, 5, 6, 9, 11, 12, 14, 15, 19, 22, 26, 28, 34, 42, 49, 68]

In [23]:
test_len = int(len(sort_case_study) * 0.2)
top = 10
worst_diag = Counter()
worst_med = Counter()
cnt = 0
for item in sort_case_study[:test_len]:
    patient = item['patient']
    for adm in patient:
        cnt += 1
        for diag in adm[0]:
            worst_diag[diag] += 1
        for med in adm[2]:
            worst_med[diag] += 1
print('adm cnt', cnt)
print('----')
for letter, count in worst_diag.most_common(top):
        print('%s: %.4f' % (letter, count))
print('----')
for letter, count in worst_med.most_common(top):
        print('%s: %.4f' % (letter, count))

adm cnt 349
----
46: 125.0000
18: 104.0000
15: 88.0000
13: 80.0000
7: 79.0000
83: 60.0000
61: 54.0000
194: 49.0000
130: 48.0000
37: 47.0000
----
46: 145.0000
414: 114.0000
49: 97.0000
187: 94.0000
117: 91.0000
37: 81.0000
7: 78.0000
41: 78.0000
4: 75.0000
77: 73.0000


In [19]:
type(sort_case_study)

list

{'ja': 0.8,
 'patient': [[[825, 7, 25],
   [65],
   [4, 6, 0, 14, 19, 1, 26, 3, 28, 39, 2, 12, 15, 87, 32]],
  [[825, 43, 24, 25],
   [602],
   [4, 6, 12, 3, 0, 14, 19, 2, 32, 39, 28, 1, 33, 15]]],
 'y_label': [[0, 1, 2, 3, 4, 6, 12, 14, 19, 32, 39],
  [0, 1, 2, 3, 4, 6, 12, 14, 15, 19, 28, 32, 39, 87]]}